<a href="https://colab.research.google.com/github/SujithaNamburu/123/blob/main/second22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install -q gradio pdfplumber sentence-transformers scikit-learn nltk


In [16]:
import nltk
nltk.download("punkt")
nltk.download("stopwords")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
from google.colab import files

uploaded = files.upload()

pdf_path = list(uploaded.keys())[0]
print("Uploaded PDF:", pdf_path)


Saving Untitled document (5).pdf to Untitled document (5).pdf
Uploaded PDF: Untitled document (5).pdf


In [18]:
import pdfplumber

def extract_pdf(pdf_path):
    pages = []
    with pdfplumber.open(pdf_path) as pdf:
        for p in pdf.pages:
            text = p.extract_text()
            if text:
                pages.append(text)
    return pages

raw_pages = extract_pdf(pdf_path)
print("Total pages extracted:", len(raw_pages))
print(raw_pages[0][:500], "...")


Total pages extracted: 10
programming_topics :
"Variables store data values in memory and can be of different types like integers, strings, and
booleans in programming languages.",
"Control structures like if-else statements allow programs to make decisions based on conditions and
execute different code paths.",
"Loops such as for and while enable repetitive execution of code blocks until a specified condition is
met or counter expires.",
"Functions are reusable blocks of code that perform specific tasks and can accept p ...


In [19]:
import numpy as np
from sentence_transformers import SentenceTransformer, util
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import re, time

stop_words = set(stopwords.words("english"))
stemmer = PorterStemmer()

# Clean text
def preprocess(text):
    text = re.sub(r"[^a-zA-Z ]", " ", text.lower())
    tokens = word_tokenize(text)
    return " ".join(stemmer.stem(t) for t in tokens if t not in stop_words and len(t) > 2)


class SearchEngine:
    def __init__(self):
        self.model = SentenceTransformer("all-MiniLM-L6-v2")
        self.pages = raw_pages
        self.page_ids = [f"Page {i+1}" for i in range(len(raw_pages))]

        # Preprocessing
        print("Preprocessing pages…")
        self.proc_pages = [preprocess(p) for p in raw_pages]

        # TF-IDF
        print("Building TF-IDF matrix…")
        self.tfidf = TfidfVectorizer(stop_words="english")
        self.tfidf_mat = self.tfidf.fit_transform(self.proc_pages)

        # BERT embeddings
        print("Generating BERT embeddings…")
        self.emb = self.model.encode(self.pages, convert_to_tensor=True, show_progress_bar=True)

    def search(self, query, method="hybrid", alpha=0.6, top_k=5):
        q_clean = preprocess(query)

        # TF-IDF
        q_vec = self.tfidf.transform([q_clean])
        t_scores = cosine_similarity(q_vec, self.tfidf_mat).flatten()

        # BERT
        q_emb = self.model.encode(query, convert_to_tensor=True)
        b_scores = util.pytorch_cos_sim(q_emb, self.emb)[0].cpu().numpy()

        if method == "tfidf":
            final = t_scores
        elif method == "bert":
            final = b_scores
        else:
            # Hybrid (normalized)
            t_norm = (t_scores - t_scores.min()) / (t_scores.max() - t_scores.min() + 1e-12)
            b_norm = (b_scores - b_scores.min()) / (b_scores.max() - b_scores.min() + 1e-12)
            final = alpha * b_norm + (1 - alpha) * t_norm

        idx = np.argsort(final)[::-1][:top_k]

        return [
            {
                "rank": i+1,
                "page": self.page_ids[j],
                "score": float(final[j]),
                "content": self.pages[j][:1000]
            }
            for i, j in enumerate(idx)
        ]


In [20]:
import nltk
nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download("stopwords")
engine = SearchEngine()
print("✓ Search engine is ready!")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Preprocessing pages…
Building TF-IDF matrix…
Generating BERT embeddings…


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✓ Search engine is ready!


In [24]:
import gradio as gr

def search_ui(query, method, alpha, top_k):
    results = engine.search(query, method=method, alpha=alpha, top_k=int(top_k))
    out = f"### Results for: `{query}`\n"
    for r in results:
        out += f"\n**{r['rank']}. {r['page']} — Score: {r['score']:.4f}**\n"
        out += r["content"] + "\n\n"
    return out


demo = gr.Interface(
    fn=search_ui,
    inputs=[
        gr.Textbox(label="Enter your search query"),
        gr.Radio(["tfidf", "bert", "hybrid"], value="hybrid"),
        gr.Slider(0, 1, value=0.6, step=0.1, label="Hybrid Alpha"),
        gr.Slider(1, 10, value=5, label="Top K")
    ],
    outputs=gr.Markdown(),
    title="🔍 PDF Search Engine (TF-IDF + BERT + Hybrid)",
    description="Upload a PDF and search inside it"
)

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8d1fb22b191eed043d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
